# UI for RT1_Assignment3

In [1]:
# import
from IPython.display import display
from move_base_msgs.msg import MoveBaseActionGoal, MoveBaseGoal
from geometry_msgs.msg import Twist
import actionlib
from actionlib_msgs.msg import GoalID
import rospy
import ipywidgets as widgets
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox, interact, interactive, fixed, interact_manual
from __future__ import print_function
from jupyros import ros3d 
import jupyros as jr
import os

In [2]:
rospy.init_node('UI_node_jupyter')

In [3]:
# Publishers definition
pub = rospy.Publisher("/move_base/goal", MoveBaseActionGoal, queue_size = 10)
pubC = rospy.Publisher("/move_base/cancel", GoalID, queue_size = 10)
pubV = rospy.Publisher("/cmd_vel", Twist, queue_size = 10)

# Variable for the goal and the cancellation
g = MoveBaseActionGoal()
gCanc = GoalID()

# Variables for the linear and angular velocity
vLin = 0.5
vAng = 1.0
vMaxLin = 2.0
vMaxAng = 4.0
vMinLin = 0.25
vMinAng = 0.5

# Variable for the modality switch (initialized 0)
# Goal publication: mod = 1
# Keyboard use: mod = 2
# Goal Cancellation: mod = 3
# Quit the program: mod = 4
mod = 0;

## Menù

#### UI widgets

In [4]:
# definition of the widgets for the position publication
xBox = widgets.BoundedFloatText(
    value = 0.0,
    min = -20.00,
    max = 20.0,
    step = 1,
    description = 'X:',
    disabled = False
)
yBox = widgets.BoundedFloatText(
    value = 0.0,
    min = -20.00,
    max = 20.0,
    step = 1,
    description = 'Y:',
    disabled = False
)
# definition of the buttons
bc = widgets.Button(
    description = 'Cancel the goal',
    disabled = False,
    button_style='',
    tootltip = 'Click me',
    icon = ''
)
bt = widgets.Button(
    description = 'PUBLISH',
    disabled = False,
    button_style='',
    tootltip = 'Click me',
    icon = ''
)


# Buttons for the keyboard drive
b_front = widgets.Button(
    description = 'FRONT',
    disabled = False,
    button_style='',
    tootltip = 'Click me',
    icon = ''
)
b_front_r = widgets.Button(
    description = 'FRONT-R',
    disabled = False,
    button_style='',
    tootltip = 'Click me',
    icon = ''
)
b_front_l = widgets.Button(
    description = 'FRONT-L',
    disabled = False,
    button_style='',
    tootltip = 'Click me',
    icon = ''
)
b_stop = widgets.Button(
    description = 'STOP',
    disabled = False,
    button_style='',
    tootltip = 'Click me',
    icon = ''
)
b_turn_l = widgets.Button(
    description = 'TURN-L',
    disabled = False,
    button_style='',
    tootltip = 'Click me',
    icon = ''
)
b_turn_r = widgets.Button(
    description = 'TURN-R',
    disabled = False,
    button_style='',
    tootltip = 'Click me',
    icon = ''
)
b_decrease = widgets.Button(
    description = 'DEC VEL',
    disabled = False,
    button_style='',
    tootltip = 'Click me',
    icon = ''
)
b_increase = widgets.Button(
    description = 'INC VEL',
    disabled = False,
    button_style='',
    tootltip = 'Click me',
    icon = ''
)
b_back = widgets.Button(
    description = 'BACK',
    disabled = False,
    button_style='',
    tootltip = 'Click me',
    icon = ''
)

# Buttons for the enable
en_goal = widgets.Button(
    description = 'Enable publication',
    disabled = False,
    button_style='',
    tootltip = 'Click me',
    icon = ''
)

en_keyboard = widgets.Button(
    description = 'Enable buttons',
    disabled = False,
    button_style='',
    tootltip = 'Click me',
    icon = ''
)

en_cancel = widgets.Button(
    description = 'Enable cancellation',
    disabled = False,
    button_style='',
    tootltip = 'Click me',
    icon = ''
)

# Definition of the rows to show
first_row = widgets.HBox([b_front_l, b_front, b_front_r])
second_row = widgets.HBox([b_turn_l, b_stop, b_turn_r])
third_row = widgets.HBox([b_back, b_increase, b_decrease])

#### Widgets' function

In [5]:
# Functions for the click button of the keyboard
def front_vel(b_front):
    global pubV
    global vLin
    global v
    v = Twist()
    v.linear.x = vLin
    v.angular.z = 0.0
    pubV.publish(v)
    
def front_r_vel(b_front_r):
    global pubV
    global vLin
    global vAng
    global v
    v = Twist()
    v.linear.x = vLin
    v.angular.z = -vAng
    pubV.publish(v)
    
def front_l_vel(b_front_l):
    global pubV
    global vLin
    global vAng
    global v
    v = Twist()
    v.linear.x = vLin
    v.angular.z = vAng
    pubV.publish(v)
    
def turn_r_vel(b_turn_r):
    global pubV
    global vAng
    global v
    v = Twist()
    v.linear.x = 0.0
    v.angular.z = -vAng
    pubV.publish(v)
    
def turn_l_vel(b_turn_l):
    global pubV
    global vAng
    global v
    v = Twist()
    v.linear.x = 0.0
    v.angular.z = vAng
    pubV.publish(v)
    
def back_vel(b_back):
    global pubV
    global vLin
    global v
    v = Twist()
    v.linear.x = -vLin
    v.angular.z = 0.0
    pubV.publish(v)
    
def stop_vel(b_stop):
    global pubV
    global v
    v = Twist()
    v.linear.x = 0.0
    v.angular.z = 0.0
    pubV.publish(v)
    
def inc_vel(b_increase):
    global vLin
    global vAng
    global vMaxLin
    global vMaxAng
    global vMinLin
    global vMinAng
    global v
    v = Twist()
    if((vLin <= vMaxLin) and (vAng <= vMaxAng)):
        vLin += 0.1*vLin
        vAng += 0.1*vAng
    else:
        vLin = vMaxLin
        vAng = vMaxAng
    
def dec_vel(b_decrease):
    global vLin
    global vAng
    global vMaxLin
    global vMaxAng
    global vMinLin
    global vMinAng
    global v
    v = Twist()
    if((vLin >= vMinLin) and (vAng >= vMinAng)):
        vLin -= 0.1*vLin
        vAng -= 0.1*vAng
    else:
        vLin = vMinLin
        vAng = vMinAng
    
# Definition of the function for the button click
# Function for the button bt to publish the position
def pub_pos(bt):
    global pub
    global g
    global mod
    
    if(mod == 1):
        g.goal.target_pose.header.frame_id = "map"
        g.goal.target_pose.pose.position.x = xBox.value
        g.goal.target_pose.pose.position.y = yBox.value
        g.goal.target_pose.pose.orientation.w = 1.0
        g.goal_id.id = "goal"
        pub.publish(g)
    
# Function for the button bc to cancel the goal
def bc_observer(bc):
    global pubC
    global gCanc
    global g
    global mod
    
    if(mod == 3):
        if(g.goal_id.id == "goal"):
            gCanc.id = g.goal_id.id
            pubC.publish(gCanc)
            g.goal_id.id = ""
        
# Function for the button en_goal
def goal_fun(en_goal):
    global mod
    mod = 1
    
# Function for the button en_cancel
def canc_fun(en_cancel):
    global mod
    mod = 3
    
# Function for the button en_keybord
def key_fun(en_keyboard):
    global mod
    mod = 2

## Publish a (x,y) position

In [6]:
# Button to enable the goal publication
display(en_goal)
en_goal.on_click(goal_fun)

# Code fot the publication with the bt button
display(widgets.VBox([xBox, yBox]))
display(bt)

bt.on_click(pub_pos)

Button(description='Enable publication', style=ButtonStyle())

Button(description='PUBLISH', style=ButtonStyle())

## Cancel the current goal

In [7]:
# Button to enable the goal cancellation
display(en_cancel)
en_cancel.on_click(canc_fun)

# Code to cancel the current goal set
display(bc)

bc.on_click(bc_observer)

Button(description='Enable cancellation', style=ButtonStyle())

Button(description='Cancel the goal', style=ButtonStyle())

## Drive by keyboard

In [8]:
# Button to enable the keyboard use
display(en_keyboard)
en_keyboard.on_click(key_fun)

# Code to drive the robot by the keyboard use
display(widgets.VBox([first_row, second_row, third_row]))

b_front.on_click(front_vel)
b_front_r.on_click(front_r_vel)
b_front_l.on_click(front_l_vel)
b_turn_r.on_click(turn_r_vel)
b_turn_l.on_click(turn_l_vel)
b_back.on_click(back_vel)
b_stop.on_click(stop_vel)
b_increase.on_click(inc_vel)
b_decrease.on_click(dec_vel)

Button(description='Enable buttons', style=ButtonStyle())

## Map

In [9]:
# Definition of the elements needed for the map
view = ros3d.Viewer()
rc = ros3d.ROSConnection(url = "ws://localhost:9090") 
tf_client = ros3d.TFClient(ros = rc, fixed_frame = 'map')
laser_view = ros3d.LaserScan(topic = "/scan", ros = rc, tf_client = tf_client) 
map_view = ros3d.OccupancyGrid(topic = "/map", ros = rc, tf_client = tf_client) 
path = ros3d.Path(topic = "/move_base/NavfnROS/plan", ros = rc, tf_client = tf_client)
urdf = ros3d.URDFModel(ros = rc, tf_client = tf_client, path = os.environ.get('JUPYROS_ASSETS_URL', 'http://localhost:3000')) 
grid = ros3d.GridModel()
view.objects = [grid, laser_view, map_view, path, urdf]

# Map visualization
#view